In [ ]:
import os

import astropy.units as u
import astropy.time
import matplotlib.pyplot as plt
import numpy as np
import sunpy.map
import sunpy_soar  # this registers the SOAR client
import pfsspy

from aiapy.calibrate import correct_degradation, update_pointing
from astropy.coordinates import SkyCoord
from matplotlib.patches import ConnectionPatch
from matplotlib.gridspec import GridSpec
from sunpy.coordinates.sun import carrington_rotation_number, carrington_rotation_time
from sunpy.net import Fido,attrs

# Affiliated Package Example

This notebook illustrates how to use several of the affiliated packages within the SunPy ecosystem in a typical solar data analysis workflow.
It also generates the components of Figure 2 in the paper.

The workflow illustrated below is comprised of the following steps:

1. Search for and download the HMI synoptic magnetogram for the appropriate Carrington rotation
2. Manually select the active region and identify the date at which that AR crossed the meridian
3. Query the EUV data for that data (using `sunpy-soar`)
4. Apply needed corrections to the EUV data (using `aiapy`)
5. Perform a field extrapolation from HMI synoptic data (using `pfsspy`)
6. Trace fieldlines and overlay them on the EUV active region observations

## Step 0

Define a few functions we will use later. These are just for convenience

In [ ]:
change_obstime = lambda x,y: SkyCoord(x.replicate(observer=x.observer.replicate(obstime=y), obstime=y))

change_obstime_frame = lambda x,y: x.replicate_without_data(observer=x.observer.replicate(obstime=y), obstime=y)

## Step 1

First, get the HMI synoptic data via the JSOC.
We know that our active region occured on the disk in the same Carrington rotation as 2022-04-01 so we will use that date to find the relevant Carrington rotation number.

In [ ]:
car_rot_date = astropy.time.Time('2022-04-01T00:00:00')
car_rot = carrington_rotation_number(car_rot_date)

In [ ]:
# When running this script, replace this with your email that you registered
# with the JSOC
jsoc_email = ...

In [ ]:
q = Fido.search(
    attrs.Time(car_rot_date, car_rot_date),
    attrs.jsoc.Series('hmi.synoptic_mr_polfil_720s'),
    attrs.jsoc.PrimeKey('CAR_ROT', int(car_rot)),
    attrs.jsoc.Notify(jsoc_email),
)

In [ ]:
q

In [ ]:
f = Fido.fetch(q, path='../data/', overwrite=True)
m_hmi = sunpy.map.Map(f)

In [ ]:
m_hmi.peek()

## Step 2

Next, we will identify the center of the AR of interest visually from the synoptic magnetogram.

In [ ]:
ar_center = SkyCoord(lon=65*u.deg, lat=15*u.deg, frame=m_hmi.coordinate_frame)

We can plot this coordinate on our map to confirm that this corresponds to a magnetically interesting region.

In [ ]:
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(111, projection=m_hmi)
m_hmi.plot(axes=ax)
ax.plot_coord(ar_center, marker='X', ls='', color='k', markersize=10)

We need to correct the obstime of the frame and of the associated observer coordinate.
This is because the default obstime of the carrington map is halfway through the carrington rotation.
However, because a synoptic map is comprised of observations from many different times, this is not the obstime for any given slice.
Thus, we look up the obstime associated with our selected longitude and use this to correct our original AR coordinate

In [ ]:
ar_date = carrington_rotation_time(int(car_rot), ar_center.lon)
ar_center_corrected = change_obstime(ar_center, ar_date)

## Step 3

Now, we will download the corresponding EUV data for this particular active region.

First, we will construct a search for data from SDO AIA and EUVI on STEREO-A.
This will search the VSO.

In [ ]:
aia_or_secchi = ((attrs.Instrument.aia | attrs.Instrument.secchi)
                 & attrs.Wavelength(171*u.angstrom)
                 & attrs.Sample(5*u.minute))

We will then construct a search query for EUI data from Solar Orbiter.
This will search the Solar Oriter Archive (SOAR) using the [`sunpy-soar` package](https://github.com/sunpy/sunpy-soar).

In [ ]:
eui_query = attrs.Level(2) & attrs.soar.Product('EUI-FSI174-IMAGE')

In [ ]:
q = Fido.search(attrs.Time(ar_date-2*u.minute, ar_date+2*u.minute),
                aia_or_secchi | eui_query)

Note that `Fido` returns three results (one for each instrument) and that two out of the three of these are from the VSO and one is from the SOAR.

In [ ]:
q

In [ ]:
files = Fido.fetch(q, path='../data/', overwrite=True)

In [ ]:
m_secchi, m_aia, m_eui = sunpy.map.Map(sorted(files))

Using the metadata provided in each file, we also note the relative locations of the three spacecraft and the unique combination of viewing angles they offer when studying this active region.

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection='polar')
# Plot the Sun
ax.plot(0, 0, marker='o', markersize=20, label='Sun', color='yellow')
# Plot the satellite locations
for m in [m_aia, m_eui, m_secchi]:
    sat = m.observatory
    coord = m.observer_coordinate
    ax.plot(coord.lon.to('rad'), coord.radius.to(u.AU), 'o', label=sat)
ax.set_theta_zero_location("S")
ax.legend(frameon=False)

## Step 4

We will now apply the neded corrections to the EUV data which include corrections for exposure time as well as instrument degradation.
We will also crop each EUV map around the active region of interest.

In the case of AIA, we'll correct the pointing keywords and also correct for the instrument degradation.
We will do this using the relevant functions from the [`aiapy` package](https://aiapy.readthedocs.io/en/stable/).

In [ ]:
m_aia = correct_degradation(update_pointing(m_aia))

The EUVI and AIA maps have also not yet been normalized for exposure time whereas the level 2 EUIV image data already have been normalized

In [ ]:
m_secchi = m_secchi / m_secchi.exposure_time
m_aia = m_aia / m_aia.exposure_time

Before cropping our full-disk maps, we can visually verify that we are cropping to the correct region.

In [ ]:
ar_width = 700*u.arcsec
ar_height = 700*u.arcsec

In [ ]:
fig = plt.figure(figsize=(15,5))
m_cutouts = []
for i,m in enumerate([m_aia,m_eui,m_secchi]):
    ax = fig.add_subplot(1,3,i+1,projection=m)
    m.plot(axes=ax,clip_interval=(1,99.99)*u.percent)
    #ax.plot_coord(ar_center_corrected, marker='o', ls='', color='C3')
    ar_center_corrected_transformed = ar_center_corrected.transform_to(m.coordinate_frame)
    blc = SkyCoord(Tx=ar_center_corrected_transformed.Tx-ar_width/2,
                   Ty=ar_center_corrected_transformed.Ty-ar_height/2,
                   frame=ar_center_corrected_transformed)
    m.draw_quadrangle(blc, width=ar_width, height=ar_height, edgecolor='C2', lw=1)

We can now crop each full disk image to the appropriate region.

In [ ]:
m_cutouts = []
for m in [m_aia, m_eui, m_secchi]:
    ar_center_corrected_transformed = ar_center_corrected.transform_to(m.coordinate_frame)
    blc = SkyCoord(Tx=ar_center_corrected_transformed.Tx-ar_width/2,
                   Ty=ar_center_corrected_transformed.Ty-ar_height/2,
                   frame=ar_center_corrected_transformed)
    # Each map is rotated prior to submapping such that the selection is aligned with the coordinate grid
    m_rot = m.rotate(missing=0.0)
    m_cutout = m_rot.submap(blc, width=ar_width, height=ar_height)
    m_cutouts.append(m_cutout)

In [ ]:
fig = plt.figure(figsize=(15,5))
for i,m in enumerate(m_cutouts):
    ax = fig.add_subplot(1,3,i+1,projection=m)
    m.plot(axes=ax,clip_interval=(1,99.99)*u.percent)

## Step 5

We will now use the [`pfsspy` package](https://pfsspy.readthedocs.io/en/stable/) to compute a potential field extrapolation solution from the HMI magnetogram we downloaded in Step 1.

We first resample the HMI synoptic map to speed up the calculation of our field extrapolation.

In [ ]:
m_hmi_resample = m_hmi.resample((1080, 540)*u.pix)

In [ ]:
nrho = 70
rss = 2.5
pfss_input = pfsspy.Input(m_hmi_resample, nrho, rss)

In [ ]:
pfss_output = pfsspy.pfss(pfss_input)

Let's plot the radial component of the field at the source surface, including the polarity inversion lines, as a check on our solution.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=pfss_output.source_surface_br)
im = pfss_output.source_surface_br.plot(axes=ax)
for pil in pfss_output.source_surface_pils:
    ax.plot_coord(pil)
fig.colorbar(im, ax=ax)

## Step 6

Finally, let's trace some fieldlines through the extrapolated field.

To begin with, we need to choose a set of seedpoints from which to trace our fieldlines.
We want to select only the seedpoints that are within a certain area around the active region.
We need to make the following transformations because we want to expressed our boundaries in terms of the active region boundary as identified by our AIA cutout.

In [ ]:
new_frame = change_obstime_frame(m_hmi.coordinate_frame, m_cutouts[0].date)
blc_ar_synop = change_obstime(m_cutouts[0].bottom_left_coord.transform_to(new_frame),
                              m_hmi.date)
trc_ar_synop = change_obstime(m_cutouts[0].top_right_coord.transform_to(new_frame),
                              m_hmi.date)

Now, mask all those points that are above -10 G and fall outside of the bounding box defined above.

In [ ]:
masked_pix_y, masked_pix_x = np.where(m_hmi_resample.data < -1e1)
seeds = m_hmi_resample.pixel_to_world(masked_pix_x*u.pix, masked_pix_y*u.pix,).make_3d()
in_lon = np.logical_and(seeds.lon > blc_ar_synop.lon, seeds.lon < trc_ar_synop.lon)
in_lat = np.logical_and(seeds.lat > blc_ar_synop.lat, seeds.lat < trc_ar_synop.lat)
seeds = seeds[np.where(np.logical_and(in_lon, in_lat))]

Now, trace fieldlines from seeds specified above

In [ ]:
ds = 0.01
max_steps = int(np.ceil(10 * nrho / ds))
tracer = pfsspy.tracing.FortranTracer(step_size=ds, max_steps=max_steps)
fieldlines = tracer.trace(SkyCoord(seeds), pfss_output,)

We also want to adjust obstime of all coordinates to coincide with AR at disk center.
By default, each field line will have the obstime of the map that the field extrapolation was computed from.
Additionally, we will choose only the close field lines.

In [ ]:
fline_coords = [change_obstime(f.coords, m_aia.date) for f in fieldlines.closed_field_lines if f.coords.shape[0]>500]

Finally, let's plot these fieldlines on top of our EUV images.

In [ ]:
fig = plt.figure(figsize=(18,5))
for i,m in enumerate(m_cutouts):
    ax = fig.add_subplot(1,3,i+1,projection=m)
    m.plot(axes=ax,title=f'{m.observatory} {m.detector} {m.wavelength.to_string(format="latex")}')
    bounds = ax.axis()
    for c in fline_coords[::5]:
        ax.plot_coord(c, lw=1, color='C2',alpha=.75)
    ax.axis(bounds)

The following code snippet builds the final version of Figure 2, but is less illustrative.

In [ ]:
# Build final figure
def add_connectors(ax1, ax2, p1, p2, color='k', lw=1):
    con1 = ConnectionPatch(
        (0, 1), ax1.wcs.world_to_pixel(p1), 'axes fraction', 'data', axesA=ax2, axesB=ax1,
        arrowstyle='-', color=color, lw=lw
    )
    con2 = ConnectionPatch(
        (1, 1), ax1.wcs.world_to_pixel(p2), 'axes fraction', 'data', axesA=ax2, axesB=ax1,
        arrowstyle='-', color=color, lw=lw
    )
    ax2.add_artist(con1)
    ax2.add_artist(con2)

h_w_ratio = 21 / 18
width = 12
frame_color = 'C3'
fig = plt.figure(figsize=(width, width*h_w_ratio))
gs = GridSpec(3, 3, figure=fig)
#Plot HMI synoptic map
ax = fig.add_subplot(gs[0,:2], projection=m_hmi)
m_hmi.plot(axes=ax, title='HMI Synoptic Magnetogram')
m_hmi.draw_quadrangle(blc_ar_synop, top_right=trc_ar_synop, color=frame_color)
ax.coords[0].grid(color='k')
ax.coords[1].grid(color='k')
# Plot spacecraft locations
ax = fig.add_subplot(gs[0,2],projection='polar')
ax.plot(0, 0, marker='o', markersize=15, label='Sun', color='yellow')
for m in m_cutouts:
    sat = m.observatory
    coord = m.observer_coordinate
    ax.plot(coord.lon.to('rad'), coord.radius.to(u.AU), 'o', label=sat)
    ax.text(coord.lon.to_value('rad')*1.15, coord.radius.to_value(u.AU)*0.95, sat)
ax.set_theta_zero_location("S")
ax.set_rlabel_position(225)
ax.set_rlim(0, 1.1)
# Plot full-disk EUV images
full_disk_axes = []
for i,m in enumerate([m_aia,m_eui,m_secchi]):
    ax = fig.add_subplot(gs[1,i],projection=m)
    m.plot(axes=ax,clip_interval=(1,99.99)*u.percent,
        title=f'{m.observatory} {m.detector} {m.wavelength.to_string(format="latex")}')
    m.draw_quadrangle(m_cutouts[i].bottom_left_coord, top_right=m_cutouts[i].top_right_coord, color=frame_color, lw=1)
    if i:
        ax.coords[1].set_axislabel(' ')
    ax.coords[0].set_axislabel(' ')
    full_disk_axes.append(ax)
    ax.coords[1].set_ticklabel(rotation=90)
# Plot EUV cutouts with fieldines
for i, m in enumerate(m_cutouts):
    ax = fig.add_subplot(gs[2,i],projection=m)
    m.plot(
        axes=ax,
        title=False,
        clip_interval=(1,99.99)*u.percent,
    )
    bounds = ax.axis()
    for c in fline_coords[::8]:
        ax.plot_coord(c, lw=1, color='C2',alpha=.75)
    ax.axis(bounds)
    if i:
        ax.coords[1].set_axislabel(' ')
    bottom_right = SkyCoord(Tx=m_cutouts[i].top_right_coord.Tx, Ty=m_cutouts[i].bottom_left_coord.Ty, frame=m_cutouts[i].coordinate_frame)
    add_connectors(full_disk_axes[i], ax, m_cutouts[i].bottom_left_coord, bottom_right, color=frame_color, lw=1)
    ax.grid(alpha=0)
    ax.coords[0].set_ticks(direction='in', color=frame_color,)
    ax.coords[1].set_ticks(direction='in', color=frame_color,)
    ax.coords[0].frame.set_color(frame_color)
    ax.coords[0].frame.set_linewidth(1)
    ax.coords[1].frame.set_color(frame_color)
    ax.coords[1].frame.set_linewidth(1)
    ax.coords[1].set_ticklabel(rotation=90)

plt.subplots_adjust(hspace=0.0)